In [1]:
import os
import math
import random
import pandas as pd
import numpy as np

import cv2
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import notebook
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn import metrics

import torch
import torch.nn as nn

from torch.nn import Parameter
from torch.nn import functional as F
from torch.utils.data import Dataset,DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
from torch.optim.lr_scheduler import _LRScheduler
from torchvision import transforms as T

from torch.utils.tensorboard import SummaryWriter
from torchvision.models import resnet101, ResNet101_Weights

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import shutil
shutil.copy('/content/drive/MyDrive/forgery_data/findit2.zip', '/content/')

'/content/findit2.zip'

In [4]:
!unzip findit2.zip

Archive:  findit2.zip
   creating: findit2/
  inflating: __MACOSX/._findit2      
   creating: findit2/test/
  inflating: __MACOSX/findit2/._test  
  inflating: findit2/train.txt       
  inflating: __MACOSX/findit2/._train.txt  
   creating: findit2/train/
  inflating: __MACOSX/findit2/._train  
  inflating: findit2/test.txt        
  inflating: __MACOSX/findit2/._test.txt  
  inflating: findit2/val.txt         
  inflating: __MACOSX/findit2/._val.txt  
   creating: findit2/val/
  inflating: __MACOSX/findit2/._val  
  inflating: findit2/test/X51007339127.png  
  inflating: __MACOSX/findit2/test/._X51007339127.png  
  inflating: findit2/test/X51006557117.txt  
  inflating: __MACOSX/findit2/test/._X51006557117.txt  
  inflating: findit2/test/X51005568884.txt  
  inflating: __MACOSX/findit2/test/._X51005568884.txt  
  inflating: findit2/test/X51006913023.png  
  inflating: __MACOSX/findit2/test/._X51006913023.png  
  inflating: findit2/test/X51006401723.png  
  inflating: __MACOSX/findit

In [5]:
class configs:
    IMAGE_SIZE = 224
    NUM_WORKERS = 2
    BATCH_SIZE = 32
    EPOCHS = 100
    SEED = 69
    CHECKPOINT = "/content/drive/MyDrive/forgery_data/checkpoints/resnet101/"
    MODEL_NAME = 'resnet101'
    scheduler_params = {
        "lr_start": 3e-5,
        "lr_max": 6e-5,
        "lr_min": 1e-5,
        "lr_ramp_ep": 5,
        "lr_sus_ep": 0,
        "lr_decay": 0.8,
    }

    model_params = {
    'num_classes':2,
    }

In [6]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [7]:
def seed_torch(seed=1):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(configs.SEED)

In [8]:
train_trns = T.Compose([
    T.ToPILImage(),
    T.Resize(size = (configs.IMAGE_SIZE,configs.IMAGE_SIZE)),
    T.RandomHorizontalFlip(p=0.5),
    T.ColorJitter(brightness=(0.8,1.2)),
    T.RandomAffine(degrees=(-10,10),translate =(0.1,0.1),shear =(-5,5,-5,5),interpolation = T.InterpolationMode.BILINEAR),
    T.RandomPerspective(distortion_scale=0.3),
    T.ToTensor(),
    T.RandomApply([T.Lambda(lambda x : x + (0.1**0.7)*torch.randn(3,configs.IMAGE_SIZE,configs.IMAGE_SIZE))],p=0.08),
    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

val_trns = T.Compose([
    T.ToPILImage(),
    T.Resize(size = (configs.IMAGE_SIZE,configs.IMAGE_SIZE)),
    T.ToTensor(),
    T.Normalize((0.445), (0.269))
])

In [9]:
class FindItData(Dataset):
    def __init__(self,df,split,transform):
        self.img_locs = list(df.image)
        self.targets = list(df.forged)
        self.transform = transform
        self.split = split
    def  __len__(self):
        return len(self.targets)
    def __getitem__(self,idx):
        img_loc = str(self.img_locs[idx])
        img = cv2.imread("./findit2/"+self.split+"/"+img_loc)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img = self.transform(img)
        target = self.targets[idx]
        return img,target

In [10]:
df_train = pd.read_csv("./findit2/train.txt")
df_test = pd.read_csv("./findit2/test.txt")

In [11]:
train_ds = FindItData(df_train,"train",train_trns)
train_loader = DataLoader(train_ds, batch_size=configs.BATCH_SIZE, shuffle=True, drop_last=True)
val_ds = FindItData(df_test,"test",val_trns)
val_loader = DataLoader(val_ds, batch_size=configs.BATCH_SIZE, shuffle=False, drop_last=True)


In [12]:
class CNNModel(nn.Module):
    def __init__(self,num_classes=2):
        super(CNNModel,self).__init__()
        self.model = resnet101(weights=ResNet101_Weights.IMAGENET1K_V1)
        self.model.fc = nn.Linear(in_features=2048, out_features=num_classes, bias=True)

    def forward(self, images):
        return self.model(images)

In [13]:
labels_tensor = torch.tensor(df_train['forged'])
class_count = torch.bincount(labels_tensor)
total_samples = torch.sum(class_count)
class_frequencies = class_count.float()/total_samples

inverse_class_weights = 1.0/class_frequencies
class_weights = inverse_class_weights / torch.sum(inverse_class_weights)

class_weights = class_weights.to(device)

In [14]:

def get_loss_acc(preds, targets):
    loss = nn.CrossEntropyLoss(weight=class_weights)(preds,targets)
    preds = torch.argmax(preds,dim=1).cpu().detach().numpy()
    targets = targets.cpu().detach().numpy()
    acc = metrics.accuracy_score(targets,preds)
    f1_score = metrics.f1_score(targets,preds)
    return loss,acc,f1_score

In [15]:
class lr_scheduler(_LRScheduler):
    def __init__(self, optimizer, lr_start=5e-6, lr_max=1e-5,
                 lr_min=1e-6, lr_ramp_ep=5, lr_sus_ep=0, lr_decay=0.8,
                 last_epoch=-1):
        self.lr_start = lr_start
        self.lr_max = lr_max
        self.lr_min = lr_min
        self.lr_ramp_ep = lr_ramp_ep
        self.lr_sus_ep = lr_sus_ep
        self.lr_decay = lr_decay
        super(lr_scheduler, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        if not self._get_lr_called_within_step:
            warnings.warn("To get the last learning rate computed by the scheduler, "
                          "please use `get_last_lr()`.", UserWarning)

        if self.last_epoch == 0:
            self.last_epoch += 1
            return [self.lr_start for _ in self.optimizer.param_groups]

        lr = self._compute_lr_from_epoch()
        self.last_epoch += 1

        return [lr for _ in self.optimizer.param_groups]

    def _get_closed_form_lr(self):
        return self.base_lrs

    def _compute_lr_from_epoch(self):
        if self.last_epoch < self.lr_ramp_ep:
            lr = ((self.lr_max - self.lr_start) /
                  self.lr_ramp_ep * self.last_epoch +
                  self.lr_start)

        elif self.last_epoch < self.lr_ramp_ep + self.lr_sus_ep:
            lr = self.lr_max

        else:
            lr = ((self.lr_max - self.lr_min) * self.lr_decay**
                  (self.last_epoch - self.lr_ramp_ep - self.lr_sus_ep) +
                  self.lr_min)
        return lr

In [16]:
class trainer:
    def __init__(self,train_dataloader,val_dataloader,load_checkpoint = False):
        if(load_checkpoint):
            print("Loading pretrained model...")
            self.model = torch.load(configs.CHECKPOINT+"model.pt")
        else:
            self.model  = CNNModel(**configs.model_params)
        no_decay = ['bias','LayerNorm.bias','LayerNorm.weight']
        para_optimizer = list(self.model.named_parameters())
        self.optimizer_parameters = [
        {'params':[p for n,p in para_optimizer if not any(nd in n for nd in no_decay)],'weight_decay':1e-5},
        {'params':[p for n,p in para_optimizer if  any(nd in n for nd in no_decay)],'weight_decay':0.0}
        ]

        self.optimizer = AdamW(amsgrad = True,params = self.optimizer_parameters,lr = 3e-5)
        self.lr_scheduler = lr_scheduler(self.optimizer,**configs.scheduler_params)
        self.criterion = get_loss_acc
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        self.model = self.model.to(device)
        self.writer = SummaryWriter(configs.CHECKPOINT+"tboard")
        self.step_train = 0
        self.step_val = 0



    def train_fn(self,epoch):
        self.model.train()
        count,total_loss,total_acc,total_f1_score = 0,0,0,0
        loop = notebook.tqdm(enumerate(self.train_dataloader),total = len(self.train_dataloader))
        for bi,data in loop:
            images = data[0].to(device)
            targets = data[1].to(device)
            preds =  self.model(images)
            self.optimizer.zero_grad()
            loss,acc,f1score =self.criterion(preds, targets)
            total_loss += loss.item()
            total_acc += acc
            total_f1_score += f1score
            self.writer.add_scalar("Training Loss",loss.item(),global_step=self.step_train)
            self.writer.add_scalar("Training Accuracy",acc,global_step=self.step_train)
            self.writer.add_scalar("Training F1 score",f1score,global_step=self.step_train)
            count +=1
            self.step_train +=1
            loss.backward()
            self.optimizer.step()
            loop.set_postfix(Epoch=epoch,Avg_Train_Loss=total_loss/count,Avg_Train_Acc=total_acc/count,Current_Train_Loss=loss.item(),Current_Train_Acc=acc,LR=self.optimizer.param_groups[0]['lr'])
        self.lr_scheduler.step()

    def eval_fn(self,epoch):
        self.model.eval()
        count,total_loss,total_acc,total_f1_score = 0,0,0,0
        with torch.no_grad():
            loop = notebook.tqdm(enumerate(self.val_dataloader),total = len(self.val_dataloader))
            for bi,data in loop:
                images = data[0].to(device)
                targets = data[1].to(device)
                preds =  self.model(images)
                loss,acc,f1score =self.criterion(preds, targets)
                total_loss += loss.item()
                total_acc += acc
                total_f1_score += f1score
                self.writer.add_scalar("Validation Loss",loss.item(),global_step=self.step_val)
                self.writer.add_scalar("Validation Accuracy",acc,global_step=self.step_val)
                self.writer.add_scalar("Training F1 score",f1score,global_step=self.step_train)
                count +=1
                self.step_val +=1
                loop.set_postfix(Epoch=epoch,Avg_Val_Loss=total_loss/count,Avg_Val_Acc=total_acc/count,Current_Val_Loss=loss.item(),Current_Val_Acc=acc)

        return total_f1_score / count

    def run(self,epochs = 5):
        best_score = 0
        for epoch in range (epochs):
            self.train_fn(epoch)
            val_score = self.eval_fn(epoch)
            print("Epoch {} complete! Validation F1 score : {}".format(epoch, val_score))
            if val_score > best_score:
                print("Best validation Score improved from {} to {}, saving model...".format(best_score, val_score))
                best_score = val_score
                torch.save(self.model, configs.CHECKPOINT+"model.pt")


In [17]:
train = trainer(train_loader,val_loader)

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 144MB/s]


In [18]:
train.run(configs.EPOCHS)

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 0 complete! Validation F1 score : 0.2647665732959851
Best validation Score improved from 0 to 0.2647665732959851, saving model...


  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 8 complete! Validation F1 score : 0.2664198832380197


  0%|          | 0/18 [00:00<?, ?it/s]

KeyboardInterrupt: ignored